# Import Libraries and Dataset

In [9]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense 
from keras.layers import Flatten 
import tensorflow as tf


# load the dataset
fashion_mnist = tf.keras.datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train,
                                                      test_size=0.25, random_state=42)

# rescale ad reshape the data
X_train, X_valid, X_test = X_train / 255.0, X_valid / 255.0, X_test / 255.0
X_train = X_train.reshape(-1, 28 * 28)
X_valid = X_valid.reshape(-1, 28 * 28)
X_test = X_test.reshape(-1, 28 * 28)

# build the model
model = Sequential([
        Flatten(input_shape=[784]),
        Dense(200, activation="relu"),
        Dense(100, activation="relu"),
        Dense(10, activation="softmax"),
])

# compile the model
model.compile(
    optimizer="sgd",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

# train the model
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid), 
          epochs=5)

Epoch 1/5
1407/1407 [==============================] - 7s 5ms/step - loss: 0.7755 - accuracy: 0.7419 - val_loss: 0.6479 - val_accuracy: 0.7638
Epoch 2/5
1407/1407 [==============================] - 4s 3ms/step - loss: 0.5127 - accuracy: 0.8240 - val_loss: 0.5200 - val_accuracy: 0.8159
Epoch 3/5
1407/1407 [==============================] - 5s 3ms/step - loss: 0.4623 - accuracy: 0.8399 - val_loss: 0.4846 - val_accuracy: 0.8323
Epoch 4/5
1407/1407 [==============================] - 4s 3ms/step - loss: 0.4350 - accuracy: 0.8489 - val_loss: 0.4422 - val_accuracy: 0.8414
Epoch 5/5
1407/1407 [==============================] - 5s 3ms/step - loss: 0.4134 - accuracy: 0.8554 - val_loss: 0.4151 - val_accuracy: 0.8535


# Save After Training Finishes




## Save and restore the entire model

In [10]:
# Save the model as a SavedModel
model.save("my_model")

INFO:tensorflow:Assets written to: my_model/assets


In [11]:
# save in the H5 format
model.save("my_model.h5")

In [12]:
my_new_model = tf.keras.models.load_model("my_model")

# evaluate the restored model
loss, accuracy = my_new_model.evaluate(X_test, y_test)
print(f"accuracy: {accuracy * 100:.2f}%")

313/313 [==============================] - 1s 2ms/step - loss: 0.4435 - accuracy: 0.8404
accuracy: 84.04%


## Save and load only the model's weights

In [13]:
model.save_weights("weights") # use `weights.h5` to save in the HDF5 format

In [14]:
from keras import Input
from keras import Model

# The two models need to share the same architecture
# I created a new model using the functional API
inputs = Input(shape=(784,))
x = Dense(200, activation="relu")(inputs)
x = Dense(100, activation="relu")(x)
outputs = Dense(10, activation="softmax")(x)
new_model = Model(inputs=inputs, outputs=outputs)

# compile the new model
new_model.compile(
    optimizer="sgd",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

# evaluate the new model before loading the weights
loss, accuracy = new_model.evaluate(X_test, y_test)
print(f"accuracy of new model before loading weights: {accuracy * 100:.2f}%")

313/313 [==============================] - 1s 2ms/step - loss: 2.3907 - accuracy: 0.0725
accuracy of new model before loading weights: 7.25%


In [15]:
# load the weights from the previous model
new_model.load_weights("weights")
# re-evaluate the new model
loss, accuracy = new_model.evaluate(X_test, y_test)
print(f"accuracy of new model after loading weights: {accuracy * 100:.2f}%")

313/313 [==============================] - 1s 2ms/step - loss: 0.4435 - accuracy: 0.8404
accuracy of new model after loading weights: 84.04%


#  Save Checkpoints During Training Using Callbacks



## Save and restore the entire model

In [16]:
from keras.callbacks import ModelCheckpoint

cp_checkpoint = ModelCheckpoint("my_model", verbose=1)

model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid), 
          epochs=5, 
          callbacks=[cp_checkpoint])

# later ...
my_model = tf.keras.models.load_model("my_model")

Epoch 1/5
1399/1407 [============================>.] - ETA: 0s - loss: 0.3969 - accuracy: 0.8617
Epoch 1: saving model to my_model
INFO:tensorflow:Assets written to: my_model/assets
1407/1407 [==============================] - 5s 3ms/step - loss: 0.3970 - accuracy: 0.8616 - val_loss: 0.4046 - val_accuracy: 0.8585
Epoch 2/5
1388/1407 [============================>.] - ETA: 0s - loss: 0.3849 - accuracy: 0.8644
Epoch 2: saving model to my_model
INFO:tensorflow:Assets written to: my_model/assets
1407/1407 [==============================] - 5s 3ms/step - loss: 0.3845 - accuracy: 0.8644 - val_loss: 0.4227 - val_accuracy: 0.8507
Epoch 3/5
1388/1407 [============================>.] - ETA: 0s - loss: 0.3711 - accuracy: 0.8691
Epoch 3: saving model to my_model
INFO:tensorflow:Assets written to: my_model/assets
1407/1407 [==============================] - 5s 3ms/step - loss: 0.3715 - accuracy: 0.8689 - val_loss: 0.4091 - val_accuracy: 0.8540
Epoch 4/5
1385/1407 [============================>.] - 

In [17]:
cp_checkpoint = ModelCheckpoint("my_model", save_best_only=True, verbose=1)

model.fit(X_train, y_train,
          validation_data=(X_valid, y_valid),
          epochs=5,
          callbacks=[cp_checkpoint])

# later...

# load the best model
my_model = tf.keras.models.load_model("my_model")

Epoch 1/5
1393/1407 [============================>.] - ETA: 0s - loss: 0.3412 - accuracy: 0.8792
Epoch 1: val_loss improved from inf to 0.40012, saving model to my_model
INFO:tensorflow:Assets written to: my_model/assets
1407/1407 [==============================] - 6s 4ms/step - loss: 0.3415 - accuracy: 0.8792 - val_loss: 0.4001 - val_accuracy: 0.8565
Epoch 2/5
1389/1407 [============================>.] - ETA: 0s - loss: 0.3342 - accuracy: 0.8814
Epoch 2: val_loss improved from 0.40012 to 0.36384, saving model to my_model
INFO:tensorflow:Assets written to: my_model/assets
1407/1407 [==============================] - 5s 4ms/step - loss: 0.3346 - accuracy: 0.8813 - val_loss: 0.3638 - val_accuracy: 0.8695
Epoch 3/5
1389/1407 [============================>.] - ETA: 0s - loss: 0.3257 - accuracy: 0.8835
Epoch 3: val_loss improved from 0.36384 to 0.36347, saving model to my_model
INFO:tensorflow:Assets written to: my_model/assets
1407/1407 [==============================] - 5s 3ms/step - loss

## Save and load only the model's weights


In [18]:
cp_checkpoint = ModelCheckpoint("weights", 
                                save_weights_only=True,
                                verbose=1)

model.fit(X_train, y_train,
          validation_data=(X_valid, y_valid), 
          epochs=5,
          callbacks=[cp_checkpoint])

Epoch 1/5
1407/1407 [==============================] - ETA: 0s - loss: 0.3067 - accuracy: 0.8898
Epoch 1: saving model to weights
1407/1407 [==============================] - 4s 3ms/step - loss: 0.3067 - accuracy: 0.8898 - val_loss: 0.3532 - val_accuracy: 0.8753
Epoch 2/5
1401/1407 [============================>.] - ETA: 0s - loss: 0.3007 - accuracy: 0.8918
Epoch 2: saving model to weights
1407/1407 [==============================] - 5s 3ms/step - loss: 0.3012 - accuracy: 0.8916 - val_loss: 0.3625 - val_accuracy: 0.8737
Epoch 3/5
1393/1407 [============================>.] - ETA: 0s - loss: 0.2956 - accuracy: 0.8942
Epoch 3: saving model to weights
1407/1407 [==============================] - 5s 3ms/step - loss: 0.2953 - accuracy: 0.8944 - val_loss: 0.3356 - val_accuracy: 0.8798
Epoch 4/5
1394/1407 [============================>.] - ETA: 0s - loss: 0.2889 - accuracy: 0.8952
Epoch 4: saving model to weights
1407/1407 [==============================] - 4s 3ms/step - loss: 0.2889 - accurac

In [19]:
# Now create a new model
inputs = Input(shape=(784,))
x = Dense(200, activation="relu")(inputs)
x = Dense(100, activation="relu")(x)
outputs = Dense(10, activation="softmax")(x)
new_model = Model(inputs=inputs, outputs=outputs, name="new_model")

# compile the new model
new_model.compile(
    optimizer="sgd",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

# and load the weights
new_model.load_weights("weights")